<a href="https://colab.research.google.com/github/yayesh16/solar-challenge-week/blob/eda-zambia/Zambia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>